# **1. introducton**
Nassau Candy is one of the largest U.S. wholesale manufacturers and distributors of specialty and private-label confections, supplying national retailers and independent stores through an extensive portfolio of 20,000+ SKUs. With vertically integrated manufacturing, nut roasting, private-label customization, and nationwide distribution, the company operates at significant scale and complexity. However, in such environments, **sales volume alone is an incomplete indicator of business success.**

High-selling products may generate weak margins, consume excessive operational cost, or mask underlying profitability issues. This analysis focuses on moving beyond revenue metrics to establish product and division level profitability clarity, enabling data-driven decision making.

Key objectives of this analysis include:

- Identifying products that truly drive gross profit and margin

- Detecting high-revenue but low-margin products that weaken overall performance

- Evaluating profitability differences across product divisions

- Highlighting margin risks to support pricing, sourcing, and portfolio optimization decisions

# **2. Problem Statement**

Despite strong sales performance across multiple product lines, Nassau Candy currently lacks clear visibility into the **true profitability drivers** of its portfolio. Decision making around pricing, promotions, and product mix is therefore constrained by limited margin-level insight.

Specifically, the organization does not have a consolidated view of:
- Which product lines deliver the highest gross margins  
- Whether high-revenue products generate proportional profit  
- How profitability and margin efficiency vary across divisions  
- Which products pose margin risk due to high cost or low efficiency  

Addressing these gaps is critical to improving financial performance and ensuring that growth is driven by **profitable, sustainable products** rather than volume alone.


# **3. Installing, Updating & Importing Libraries**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline

# **4. Data Acquisition & Description**

### **4.1 Dataset Fields Description (Data Dictionary):**
| Field | Description |
|------|-------------|
| Row ID | Unique row identifier |
| Order ID | Unique order identifier |
| Order Date | Date of order |
| Ship Date | Date of shipment |
| Ship Mode | Shipping method of order |
| Customer ID | Unique customer identifier |
| Country/Region | Country or region of customer |
| City | City of customer |
| State/Province | State/province of customer |
| Postal Code | Postal code / zip code of customer |
| Division | Product division |
| Region | Region of customer |
| Product ID | Unique product identifier |
| Product Name | Product long namer |
| Sales | Total sales value of order |
| Units | Total units of order |
| Gross Profit | Gross profit of order ( Sales - Cost ) |
| Cost | Cost to manufacture |

### **4.2 Load Dataset**

In [4]:
# Loading the dataset into a dataframe
data=pd.read_csv('https://raw.githubusercontent.com/amrit1426/Unified_mentor_project/main/Nassau_Candy_Distributor.csv')

### **4.3 Data Initial Inspection**
- In this section we will get **information about the data** and see some observations.

In [5]:
data.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State/Province,Postal Code,Division,Region,Product ID,Product Name,Sales,Units,Gross Profit,Cost
0,1,US-2021-103800-CHO-MIL-31000,03-01-2024,30-06-2026,Standard Class,103800,United States,Houston,Texas,77095,Chocolate,Interior,CHO-MIL-31000,Wonka Bar - Milk Chocolate,6.500,2,4.220,2.280
1,2,US-2021-112326-CHO-TRI-54000,04-01-2024,01-07-2026,Standard Class,112326,United States,Naperville,Illinois,60540,Chocolate,Interior,CHO-TRI-54000,Wonka Bar - Triple Dazzle Caramel,7.500,2,4.900,2.600
2,3,US-2021-112326-CHO-NUT-13000,04-01-2024,01-07-2026,Standard Class,112326,United States,Naperville,Illinois,60540,Chocolate,Interior,CHO-NUT-13000,Wonka Bar - Nutty Crunch Surprise,10.470,3,7.470,3.000


In [6]:
print('Dataset Shape (Rows, Cols):', data.shape)

Dataset Shape (Rows, Cols): (10194, 18)


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          10194 non-null  int64  
 1   Order ID        10194 non-null  object 
 2   Order Date      10194 non-null  object 
 3   Ship Date       10194 non-null  object 
 4   Ship Mode       10194 non-null  object 
 5   Customer ID     10194 non-null  int64  
 6   Country/Region  10194 non-null  object 
 7   City            10194 non-null  object 
 8   State/Province  10194 non-null  object 
 9   Postal Code     10194 non-null  object 
 10  Division        10194 non-null  object 
 11  Region          10194 non-null  object 
 12  Product ID      10194 non-null  object 
 13  Product Name    10194 non-null  object 
 14  Sales           10194 non-null  float64
 15  Units           10194 non-null  int64  
 16  Gross Profit    10194 non-null  float64
 17  Cost            10194 non-null 

In [8]:
data.describe()

,Row ID,Customer ID,Sales,Units,Gross Profit,Cost
count,10194.000,10194.000,10194.000,10194.000,10194.000,10194.000
mean,5097.500,134468.961,13.909,3.792,9.166,4.742
std,2942.899,20231.483,11.341,2.228,6.644,5.062
min,1.000,100006.000,1.250,1.000,0.250,0.600
25%,2549.250,117212.000,7.200,2.000,4.900,2.400
50%,5097.500,133550.000,10.800,3.000,7.470,3.600
75%,7645.750,152051.000,18.000,5.000,12.250,5.700
max,10194.000,192314.000,260.000,14.000,130.000,130.000


#### **Observation:**
- The dataset contains transactional-level sales, cost, and profitability information for Nassau Candy’s product portfolio. Each row represents a single product-level order record.
- Dataset Shape is **(10194, 18)**
- At a high level, we can observe that there are **no missing values** in our dataset.
- **No zero or negative values** were observed in the numerical columns.
- Some columns have **incorrect data types**

# **5. Data Cleaning & Validation**
Analytical Methodology (Step-by-Step)
Data Cleaning & Validation
- Validate cost and sales values
- Remove zero-sales or invalid profit records
- Handle missing unit values
- Standardize product and division labels

In [10]:
# Copying pimary dataframe in a temporary variable for further analysis
df=data.copy()

### **5.1 Structural Integrity Checks**
In this section, structural integrity checks are performed to ensure the dataset is suitable for analysis. The following validations are conducted:
- Duplicate record check

- Missing value check

In [11]:
# Checking for duplicate rows
print('Total number of duplicate rows:',df.duplicated().sum())

Total number of duplicate rows: 0


In [12]:
# Checking for missing values
df.isna().sum()

,0
Row ID,0
Order ID,0
Order Date,0
Ship Date,0
Ship Mode,0
Customer ID,0
Country/Region,0
City,0
State/Province,0
Postal Code,0


### **5.2 Data Type Standardization**

To ensure accurate analysis and proper time-based operations, the following data type conversions were performed:

- Converted Order Date to datetime format for chronological analysis and time-series aggregation.
- Converted Ship Date to datetime format to enable shipment duration and delivery trend analysis.
- Cast Customer ID to object type to preserve its categorical nature and prevent unintended numerical operations.

In [13]:
# Converting columns to appropiate dtypes
df['Order Date'] = pd.to_datetime(df['Order Date'],format='%d-%m-%Y')
df['Ship Date'] = pd.to_datetime(df['Ship Date'],format='%d-%m-%Y')
df['Customer ID'] = df['Customer ID'].astype(object)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          10194 non-null  int64         
 1   Order ID        10194 non-null  object        
 2   Order Date      10194 non-null  datetime64[ns]
 3   Ship Date       10194 non-null  datetime64[ns]
 4   Ship Mode       10194 non-null  object        
 5   Customer ID     10194 non-null  object        
 6   Country/Region  10194 non-null  object        
 7   City            10194 non-null  object        
 8   State/Province  10194 non-null  object        
 9   Postal Code     10194 non-null  object        
 10  Division        10194 non-null  object        
 11  Region          10194 non-null  object        
 12  Product ID      10194 non-null  object        
 13  Product Name    10194 non-null  object        
 14  Sales           10194 non-null  float64       
 15  Un

### **5.3 Financial Data Validation**
In this section the following integrity checks are performed to ensure the dataset is suitable for analysis:

- Sales value validation
- Cost value validation
- Unit quantity validation
- Gross Profit consistency check (Sales − Cost)

In [15]:
# Data validation checks
print(f"Transactions with zero or negative Sales: {(df['Sales'] <= 0).sum()}")
print(f"Transactions with zero or negative Cost: {(df['Cost'] <= 0).sum()}")
print(f"Transactions with zero or negative Units: {(df['Units'] <= 0).sum()}")
print(f"Transactions with zero or negative Gross Profit: {(df['Gross Profit'] <= 0).sum()}")
print(f"Transactions with Gross Profit mismatch (Sales - Cost): "
      f"{(df['Gross Profit'].round(2) != (df['Sales'] - df['Cost']).round(2)).sum()}")

Transactions with zero or negative Sales: 0
Transactions with zero or negative Cost: 0
Transactions with zero or negative Units: 0
Transactions with zero or negative Gross Profit: 0
Transactions with Gross Profit mismatch (Sales - Cost): 0


### **5.4 Standardize product and division labels**
In this step, product and division labels were standardized to ensure consistency across categorical fields:
- Leading and trailing whitespace was removed from Product Name and Division fields

- Text casing was normalized to a consistent title case format

- Unique values were reviewed to confirm successful standardization and to detect any residual inconsistencies

In [16]:
# Strip extra spaces and normalize casing
df['Division'] = df['Division'].str.strip().str.title()
df['Product Name'] = df['Product Name'].str.strip().str.title()

In [17]:
# Checking unique values to confirm
print('Product Divisions:', df['Division'].unique())
print('\nProduct Names:', df['Product Name'].unique())

Product Divisions: ['Chocolate' 'Other' 'Sugar']

Product Names: ['Wonka Bar - Milk Chocolate' 'Wonka Bar - Triple Dazzle Caramel'
 'Wonka Bar - Nutty Crunch Surprise' 'Wonka Bar -Scrumdiddlyumptious'
 'Wonka Bar - Fudge Mallows' 'Wonka Gum' 'Kazookles' 'Lickable Wallpaper'
 'Fizzy Lifting Drinks' 'Laffy Taffy' 'Sweetarts' 'Nerds' 'Hair Toffee'
 'Everlasting Gobstopper' 'Fun Dip']


#### **Observation:**
- No duplicate records or missing values were identified in the dataset.
- Incorrect data types have been standardized.
- All financial integrity checks passed successfully. No transactions with zero or negative sales, cost, units, or gross profit were identified. Additionally, gross profit values were found to be fully consistent with the calculated difference between sales and cost (after rounding for currency precision).
- Product and division labels have been standardized.